In [2]:
from bs4 import BeautifulSoup
import requests
import urllib.request

from scipy import ndarray
import skimage as sk
from skimage import transform
from skimage import util

import numpy as np
import random 

from PIL import Image
import cv2

from pathlib import Path
import os
import glob

import torch
from torch import nn
from torch import optim

In [13]:
def fix_random_seed(seed):
    """Ensure reproducible results"""

    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

seed = 1
fix_random_seed(seed)

In [3]:
# Get information from page of all minions
page = requests.get("https://despicableme.fandom.com/wiki/Category:Minions?from=A")
page

<Response [200]>

In [4]:
# Get a list of HREFs for each minion page
soup = BeautifulSoup(page.content, 'html.parser')
minions = soup.find_all('a', class_="category-page__member-link")


In [5]:
# Get links to page of each minion
def get_minion_page(minion_page_link):
  src = "https://despicableme.fandom.com/"
  return src + minion_page_link.get('href')

minion_links = map(get_minion_page, minions)  
minion_links = list(minion_links)



In [6]:
def make_dir(directory):
  Path(directory).mkdir(parents=True, exist_ok=True)

In [7]:
# Create a folder for minion images and augmented images
make_dir("/content/minion_imgs")
make_dir("/content/augmented_images/")

def get_title_from_path(fname):
  start = fname.find("/")
  end = fname.find(".")
  return fname[start+1:end]

In [8]:
# For each Minion page, get Minion thumbnail
for page in minion_links:

  page = requests.get(page)
  soup = BeautifulSoup(page.content, 'html.parser')

  #Thumbnail Link
  img_link = soup.find('a', class_="image image-thumbnail")
  
  if img_link is None:
    continue
  
  img_link = img_link.get('href')
  
  #Image title
  title = soup.find('h1', class_="page-header__title")
  title = title.get_text().strip()
  
  save_dir = "/content/minion_imgs/" + title + ".jpg"

  urllib.request.urlretrieve(img_link, save_dir)

In [46]:
for dir in glob.glob("minion_imgs/*"):
  try:
    im = Image.open(dir)
    im.resize((600, 400), Image.BICUBIC)
    title = get_title_from_path(dir)
    im.save("/content/minion_imgs/" + title + ".png")
    os.remove("/content/minion_imgs/" + title + ".jpg")
  except:
    os.remove(dir)
  
   

In [10]:
#Image Transofrmation functions

def get_rand_binary():
  return random.choice([0, 1])

def random_rotation(image_array: ndarray):
    # pick a random degree of rotation between 25% on the left and 25% on the right
    random_degree = random.uniform(-25, 25)
    return sk.transform.rotate(image_array, random_degree)

def add_random_noise(image_array: ndarray):
    # add random noise to the image
    return sk.util.random_noise(image_array)

def horizontal_flip(image_array: ndarray):
    # horizontal flip doesn't need skimage, it's easy as flipping the image array of pixels !
    return image_array[:, ::-1]
    
def augment_image(image_array: ndarray):
  im = image_array
  
  rotate_im = get_rand_binary()
  if rotate_im:
    im = random_rotation(im)
  
  add_noise = get_rand_binary()
  if add_noise:
    im = add_random_noise(im)
  
  flip_image = get_rand_binary()
  if flip_image:
    im = horizontal_flip(im)
  
  return im



In [11]:
# Create list containing numpy representations of the images and their titles

image_list = []
image_titles = []

def get_title_from_path(fname):
  start = fname.find("/")
  end = fname.find(".")
  return fname[start+1:end]

for filename in sorted(glob.glob('minion_imgs/*.png')):
  im=Image.open(filename)
  image_list.append(np.asarray(im))
  im_title = get_title_from_path(filename)
  image_titles.append(im_title)

print(image_titles)

['Barry', 'Bill (Training wheels)', 'Bob (Despicable Me)', 'Bob (orientation day)', 'Bob', 'Brian', 'Buck', 'Carl (Despicable Me)', 'Carl', 'Charlie', 'Darwin', 'Dave (Minions)', 'Dave', 'Disco Minion Costume', 'Donnie', 'Donny (Despicable Me 2)', 'Evil Minions', 'Fred (Minions)', 'Jerry', 'Joe', 'John', 'Jon', 'Jorge', 'Ken (Minions Paradise)', 'Ken (Minions)', 'Ken', 'Kevin (Orientation Day)', 'Kevin', 'Lance', 'Larry', 'Lewis', 'Mack', 'Mark', 'Mel', 'Mike', 'Minions', 'Norbert', 'Otto', 'Partier Minion Costume', 'Paul (Minion Rush)', 'Paul', 'Peter', 'Phil (Despicable Me)', 'Phil', 'Steve', 'Stuart', 'The Jelly Jar Minion', 'Tim', 'Tom', 'Traveling Sales Minion']


In [12]:
# Create augmented images and add them to folder

num_skews_per_original = 5

def get_title(image_title, skew_no):
  num = str(skew_no)
  
  if skew_no < 10:
    num = "0" + num
    
  return image_title + "_" + num + ".png"

for j, image in enumerate(image_list):
  for i in range(num_skews_per_original):

    augmented_im = augment_image(image)
   
    augmented_im = Image.fromarray((augmented_im * 255).astype(np.uint8))
    title = get_title(image_titles[j], i)
    augmented_im.save("/content/augmented_images/"+ title)
    


In [16]:
def batch_order(num_samples, shuffle=True):
  shuffle_order = np.arange(num_augmentations)
  if shuffle:
    random.shuffle(shuffle_order)
  return shuffle_order

num_augmentations = len(glob.glob('augmented_images/*.png'))
shuffle_samples = True # Set to true to ensure that batches have augmentations from multiple different minions

shuffle_order = batch_order(num_augmentations, shuffle=shuffle_samples)

def get_num_batches(fname, batch_size):
  images = glob.glob(fname+"*")
  num_images = len(images)
  return num_images / batch_size if num_images % batch_size == 0 else num_images / batch_size + 1


def get_images_from_folder(fname):
  images = glob.glob(fname+"*")
  num_images = len(images)
  return get_images_from_folder_batch(fname, num_images, 1)

def get_images_from_folder_batch(fname, batch_size, batch_no):
  image_arr = []

  images = np.array(glob.glob(fname+"*"))
  start = (batch_no-1)*batch_size
  end = min(batch_no*batch_size, len(images))

  indices = np.array(shuffle_order[start:end]).astype(int)
  if start >= len(images):
    return image_arr

  batch = images[indices]

  for im in batch:
    image_arr.append(np.asarray(Image.open(im)))

  return image_arr
# Load augmented images from folder, add to list
augmented_images = get_images_from_folder_batch("/content/augmented_images/", 20, 1)


In [131]:
class Generator(nn.Module):
  def __init__(self, nz=100, ngf=500, nc=400):
    super().__init__()
    # Other initialization stuff, add any nn sequentials 
    self.cnn1 = nn.Sequential(
      nn.ConvTranspose2d( nz, ngf * 8, 4, 1, 0, bias=False),
      nn.BatchNorm2d(ngf * 8),
      nn.ReLU(True),
      nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
      nn.BatchNorm2d(ngf * 4),
      nn.ReLU(True),
      nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
      nn.BatchNorm2d(ngf * 2),
      nn.ReLU(True),
      nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),
      nn.BatchNorm2d(ngf),
      nn.ReLU(True),
      nn.ConvTranspose2d( ngf, nc, 4, 2, 1, bias=False),
      nn.Tanh()
  )
    
    self.cnn = nn.Sequential(
      nn.ConvTranspose2d( nz, ngf * 8, 4, 1, 0, bias=False),
      nn.BatchNorm2d(ngf * 8),
      nn.ReLU(True),
      nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
      nn.BatchNorm2d(ngf * 4),
      nn.ReLU(True),
      nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
      nn.BatchNorm2d(ngf * 2),
      nn.ReLU(True),
      nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),
      nn.BatchNorm2d(ngf),
      nn.ReLU(True),
      nn.ConvTranspose2d( ngf, nc, 4, 2, 1, bias=False),
      nn.Tanh()
  )
  
    def init_weights(m):
      if isinstance(m, nn.ConvTranspose2d):
          torch.nn.init.xavier_normal_(m.weight, gain=10)
          
    self.cnn.apply(init_weights)
    self.cnn1.apply(init_weights)

  def forward(self, init_vec):
    
    generated = self.cnn(init_vec)
    print(generated.shape)

    return generated

In [60]:
class Descriminator(nn.Module):
  def __init__(self):
    super().__init__()
    # Other initialization stuff, add any nn sequentials here

    self.init_weights(nn.init.kaiming_uniform_)

  def init_weights(self, init_fn):
    def init(m):
      for child in m.children():
        init_fn(child.weights)
    init(self)    

  def forward(self, real_data, fake_data):
    return 

In [132]:
n_epochs = 10
generator = Generator()
generator.eval()
response = generator(torch.ones((1,100, 1, 1)))
im=Image.open("/content/augmented_images/Barry_00.png")
im=np.asarray(im)
print(im.shape)
print(response.shape)
response = response.detach().numpy()
print((response[0] * 255).astype(np.uint8))
augmented_im.save("/content/test.png")

# Step 1: Train Descriminator on real data for n epochs
# Step 2: Generate fake inputs for generator and train discriminator on fake data

torch.Size([1, 400, 64, 64])
(790, 756, 3)
torch.Size([1, 400, 64, 64])


TypeError: ignored